# Honeyword Encryption fdasf

**Plan projektu:**

1. historia dobrze wykorzystanej przynęty
2. wstępne informacje o temacie
3. implementacja w menagera haseł

### 1. Operacja Micemeat

30 kwietnia 1943 roku na hiszpańskim wybrzeżu zostało znalezione ciało majora Royal Marines przez tamtejszego rybaka. Pomimo neutralności Hiszpanii w okolicy znajdował się niemiecki posterunek, którego agent zainteresował się wrogimi zwłokami. Zmarły posiadał przy sobie dokumenty, które okazały się dla niezwykle cenne dla Abwehry - zawierały informacje o planowanej dużej inwazji aliantów na **Grecję**. Niemcy przypuszczali wystąpienie inwazji, ale brakowało im informacji o miejscu. Przechowony list skutkwał rozkazem Führera, który przemieścił trzy pancerne dywizje na wybrzeża Grecji.

<details>
<br>
Inwazja została przeprowadzona na Sycylię. Znaleziony major nigdy nie istniał, jego tożsamość została stworzona na potrzeby operacji przez brytyjski wywiad. Oczywiście celem było wprowadzenie wroga w błąd.
<br>
<br>
Podczas fabrykacji żołnierza dołożono wszelkich starań dla zagwarantowania autentyczności - przy jego ciele znajdowały się też zdjęcie abstrakcyjnej narzeczonej, listy niepocieszonego ojca, list wychwalający majora jako eksperta w dziedzinie operacji desantowych, prawdziwe bilety z kina, ale również nota z banku o przekroczeniu stanu konta, niezapłacony rachunek i przepustka, która straciła ważność.
<br>
Ponadto na łamach dziennika The Times opublikowano nazwisko majora na liście ofiar brytyjskich. Do władz Hiszpanii wysłano ciąg pilnych depesz, w których brytyjczycy upominali się o zwrot dokumentów (z należytym zachowaniem dyskrecji). Dzięki odpowiednim przygotowaniu dokumentów, po ich odesłaniu brytyjscy specjaliści byli w stanie potwierdzić, że zostały one wcześniej otwarte.
<br>
<br>
Powyższe działania pozwoliły na względnie niewielki opór przeciwnika podczas inwazji. Co więcej miały też wpływ na późniejsze operacje - Niemcy przestali wierzyć znajdowanym dokumentom.
</details>

In [54]:
import random
import re
import sys
from typing import Counter

random.seed(10)
n = 256 # wielkości przestrzeni seedów

# Implementacja oraz jej objaśnienie
![](src/honey_mapowanie.png)
## Uwagi
W tej demonstracji pomijamy etap generowania przestrzeni wiadomości ponieważ jak pisaliśmy wcześniej zagadnienie to wykracza ponad zakres omawianego materiału oraz jest mocno specyficzne dla każdego rodzaju szyfrowanych wiadomości. Tutaj posłużymy sie 100 najpopularniejszymi hasłami i założymy ze szyfrowane hasło jest jednym z listy. W przypadku pełnego honey encryption przestrzeń wiadomości zostałaby wygenerowana dla danej szyfrowanej wiadomości i bezpieczeństwo szyfru byłoby uzależnione od jakości wygenerowanej przestrzeni wiadomości oraz obliczonego prawdopodobieństwa dla każdej z nich. Korzystając z naszego rozwiązania prawdopodobieństwo jest wyznaczane empirycznie z bazy najpopularniejszych haseł więc jest jak praktycznie idealne. Poniżej przedstawione sa funkcje do parsowania bazy wiadomości oraz obliczenia prawdopodobieństwa wystąpienia danego hasła (zakładając ze występują wyłącznie hasła z listy).

In [55]:

def parse_passwd_list(path):
    p_list = list()
    count_pattern = re.compile(r'\.\s+(\d+)')
    password_pattern = re.compile(r'\b\w+$')
    total_count = 0

    with open(path) as f:
        for line in f.readlines():
        
            count = count_pattern.search(line)
            passwd = password_pattern.search(line)

            if passwd and count:
                count = int(count.group(1))
                p_list.append([passwd.group(), count])
                total_count += count
    
    return p_list, total_count


def get_prob_list(p_list, total_count):
    return [(passwd, count/total_count) for passwd, count in p_list]

p_list, total_count = parse_passwd_list('common_passwords.txt')
prob_list = get_prob_list(p_list, total_count)
print('Lista (hasło, prawdopodobieństwo wystąpienia)\n', prob_list)



Lista (hasło, prawdopodobieństwo wystąpienia)
 [('123456', 0.16876664151602264), ('123456789', 0.08234487145110903), ('password', 0.07188386638148746), ('adobe123', 0.04399471212905398), ('12345678', 0.04189972583719427), ('qwerty', 0.027194289764519302), ('1234567', 0.02582680144086169), ('111111', 0.023671536745922372), ('photoshop', 0.017337523721630176), ('123123', 0.017188490566429918), ('1234567890', 0.015986248209835355), ('000000', 0.015835760058698693), ('abc123', 0.014714373904855737), ('1234', 0.012773409325692526), ('adobe1', 0.011794612773617182), ('macromedia', 0.011359568988632324), ('azerty', 0.010153793070477925), ('iloveyou', 0.009798774061995297), ('aaaaaa', 0.009204096437130665), ('654321', 0.009077096077538812), ('12345', 0.009041136892253394), ('666666', 0.007775290427581735), ('sunshine', 0.007342533064782655), ('123321', 0.007267288989214323), ('letmein', 0.006953217723513358), ('monkey', 0.0067655232477172155), ('asdfgh', 0.006558705968185474), ('password1', 0.

### Funkcja `get_numbered_list(prob_list, n)` 
Oblicza do ilu seed'ów będzie mapowana dana wiadomość na podstawie wielkości przestrzeni seedów oraz prawdopodobieństwa wystąpienia danej wiadomości, zwraca posortowana listę

In [56]:

def get_numbererd_list(prob_list, n):
    n_list = [[passwd, round(prob * n)] for passwd, prob in prob_list]
    n_list = sorted(n_list, key=lambda x: x[1], reverse=True)
    s = sum(map(lambda x: x[1], n_list))

    diff = s - n
    l = len(n_list)

    if diff > 0:
        for i in range(diff):
            n_list[i][1] -= 1
    elif diff < 0:
        for i in range(abs(diff)):
            n_list[l-i-1][1] += 1

    return sorted(n_list, key=lambda x: x[1], reverse=True)

n_list = get_numbererd_list(prob_list, n)

print('Lista hasło, do ilu seedów jest mapowane)\n', n_list)



Lista hasło, do ilu seedów jest mapowane)
 [['123456', 43], ['123456789', 21], ['password', 18], ['adobe123', 11], ['12345678', 11], ['qwerty', 7], ['1234567', 7], ['111111', 6], ['photoshop', 4], ['123123', 4], ['1234567890', 4], ['000000', 4], ['abc123', 4], ['1234', 3], ['adobe1', 3], ['macromedia', 3], ['azerty', 3], ['iloveyou', 3], ['aaaaaa', 2], ['654321', 2], ['12345', 2], ['666666', 2], ['sunshine', 2], ['123321', 2], ['letmein', 2], ['monkey', 2], ['asdfgh', 2], ['password1', 2], ['shadow', 2], ['secret', 2], ['summer', 2], ['1q2w3e', 2], ['snoopy1', 2], ['princess', 1], ['dragon', 1], ['adobeadobe', 1], ['daniel', 1], ['computer', 1], ['michael', 1], ['121212', 1], ['charlie', 1], ['master', 1], ['superman', 1], ['qwertyuiop', 1], ['112233', 1], ['asdfasdf', 1], ['jessica', 1], ['1q2w3e4r', 1], ['welcome', 1], ['1qaz2wsx', 1], ['987654321', 1], ['fdsa', 1], ['753951', 1], ['chocolate', 1], ['fuckyou', 1], ['soccer', 1], ['tigger', 1], ['asdasd', 1], ['thomas', 1], ['asdfghjk

### Funkcja `get_mapping_message_to_seed(n_list, n)`
Zwraca mapowanie wiadomości do poszczególnych seedów. tutaj posłużyliśmy sie przestrzenią 256 seedów każdy z nich można traktować jako jeden bajt.

In [57]:
def get_mapping_message_to_seed(n_list, n):
    nums = [x for x in range(n)]
    random.shuffle(nums)
    i = 0
    message_to_seed = list()

    for passwd, num in n_list:
        mess_map = list()
        for _ in range(num):
            mess_map.append(nums[i])
            i += 1
        message_to_seed.append([passwd, mess_map])

    return message_to_seed

message_to_seed = get_mapping_message_to_seed(n_list, n)
print('Lista [hasło, [seedy do których jest mapowane]]\n',message_to_seed)

Lista [hasło, [seedy do których jest mapowane]]
 [['123456', [156, 130, 191, 213, 15, 134, 227, 138, 47, 165, 246, 160, 91, 252, 73, 46, 141, 166, 32, 2, 26, 205, 144, 50, 39, 204, 131, 100, 102, 249, 135, 7, 151, 171, 224, 235, 199, 82, 29, 76, 184, 216, 27]], ['123456789', [163, 65, 253, 85, 22, 132, 225, 226, 23, 182, 75, 79, 10, 176, 250, 209, 254, 230, 201, 177, 5]], ['password', [94, 223, 31, 119, 37, 14, 251, 239, 25, 101, 242, 217, 200, 78, 129, 189, 54, 87]], ['adobe123', [33, 43, 142, 64, 55, 228, 194, 108, 58, 124, 13]], ['12345678', [237, 195, 143, 6, 146, 161, 104, 0, 4, 117, 12]], ['qwerty', [56, 245, 121, 162, 192, 203, 152]], ['1234567', [148, 74, 180, 53, 17, 51, 178]], ['111111', [59, 86, 219, 21, 181, 236]], ['photoshop', [150, 187, 66, 69]], ['123123', [183, 126, 99, 186]], ['1234567890', [103, 159, 222, 174]], ['000000', [158, 81, 84, 248]], ['abc123', [139, 206, 185, 193]], ['1234', [188, 88, 122]], ['adobe1', [229, 231, 164]], ['macromedia', [179, 214, 68]], ['az

### Funkcja `get_mapping_seed_to_message(message_to_seed, n)` 
Zwraca odwrotne mapowanie do funkcji powyżej. To jest do jakiej wiadomości mapuje sie każdy seed.

In [58]:
def get_mapping_seed_to_message(message_to_seed, n):
    seed_to_message = [[x] for x in range(n)]
    for passwd, mess_map in message_to_seed:
        for i in mess_map:
            seed_to_message[i].append(passwd)

    return seed_to_message

seed_to_message = get_mapping_seed_to_message(message_to_seed, n)
print('Lista [seed, hasło do którego sie mapuje]\n', seed_to_message)

Lista [seed, hasło do którego sie mapuje]
 [[0, '12345678'], [1, '987654321'], [2, '123456'], [3, 'asdfghj'], [4, '12345678'], [5, '123456789'], [6, '12345678'], [7, '123456'], [8, 'killer'], [9, 'secret'], [10, '123456789'], [11, 'abc'], [12, '12345678'], [13, 'adobe123'], [14, 'password'], [15, '123456'], [16, 'matrix'], [17, '1234567'], [18, 'asdfgh'], [19, '123123123'], [20, 'monkey'], [21, '111111'], [22, '123456789'], [23, '123456789'], [24, '666666'], [25, 'password'], [26, '123456'], [27, '123456'], [28, 'sunshine'], [29, '123456'], [30, 'azerty'], [31, 'password'], [32, '123456'], [33, 'adobe123'], [34, 'asdfghjkl'], [35, 'buster'], [36, 'iloveyou'], [37, 'password'], [38, 'monkey'], [39, '123456'], [40, '1q2w3e'], [41, 'abcdef'], [42, '12345'], [43, 'adobe123'], [44, 'zxcvbnm'], [45, 'azerty'], [46, '123456'], [47, '123456'], [48, '12345'], [49, '1q2w3e4r'], [50, '123456'], [51, '1234567'], [52, 'samsung'], [53, '1234567'], [54, 'password'], [55, 'adobe123'], [56, 'qwerty'], 

### Funkcja `encode(message, message_to_seed)` 
Oblicza seed dla wiadomości z przestrzeni seedów na postawie przekazanego mapowania zwracając losowy element z listy seedów do których mapuje sie dane hasło. Tutaj wybraliśmy hasło `freedom` ponieważ dla przestrzeni 256 seedów jest ono mapowane do wyłącznie jednego co nie pozwala na przypadkowe zwrócenie hasła dla złego klucza (w dalszej części dokładniejsze omówienie tego problemu)

In [87]:
def encode(message, message_to_seed):
    i = list(map(lambda x: x[0], message_to_seed)).index(message)
    return random.choice(message_to_seed[i][1])

message = 'freedom'
encoded_message = encode(message, message_to_seed)
print(f'seed dla hasła "{message}" to "{encoded_message} a binarnie {bin(encoded_message)}"')

seed dla hasła "freedom" to "118 a binarnie 0b1110110"


### Funkcja `decode(seed, seed_to_message)` 
Zwraca odkodowana wiadomość dal danego seeda (każdy seed mapuje sie do jednego hasła) na podstawie przekazanego mapowania.

In [88]:
def decode(seed, seed_to_message):
    return seed_to_message[seed][1]

decoded_message = decode(encoded_message, seed_to_message)
print(f'seed "{encoded_message}" mapuje sie do hasła "{decoded_message}"')

seed "118" mapuje sie do hasła "freedom"


### Funkcja `encode_password(password, n)` 
Przekształca dowolny klucz do przestrzeni seedów. Tutaj jest to rozwiązane w bardzo prosty sposób na podstawie sprowadzenia klucza do wartości numerycznej oraz obliczenia mod n gdzie n = 256. To sprowadza dowolny klucz do bajta niestety dla tak malej przestrzeni seedów oraz prymitywnej funkcji otrzymamy szybko kolizje co w konsekwencji zwróci poprawne hasło dla niepoprawnego klucza. Jednak na potrzeby demonstracji i dla przejrzystości kodu zdecydowaliśmy sie na tak małą przestrzeń.

In [99]:
def encode_password(password, n):
    return hash(password) % n

passwd = 'kryptografia'
encoded_password = encode_password(passwd, n)
print(f'dla klucza "{passwd}" po przekształceniu otrzymamy seed "{encoded_password} a binarnie {bin(encoded_password)}"')

dla klucza "kryptografia" po przekształceniu otrzymamy seed "97 a binarnie 0b1100001"


### Funkcja `encrypt(message, password, message_to_seed, n)` 
Korzystając z funkcji przestawionych powyżej zwraca szyfrogram który w przypadku honey encryption należy do przestrzeni seedów. Najpierw podane hasło zamieniane jest w element z przestrzeni seedów (`encode_password`) a następnie wiadomość która ma byc zakodowana równie jest przekształcana (`encode`). Kolejnym krokiem jest operacja XOR miedzy seedem dla wiadomości oraz hasła. Wynik jest zwracany jako szyfrogram.
### Przykład użycia:
Jako klucz kodujący hasło wybieramy `kryptografia`
hasło które chcemy zakodować to `freeedom` dlatego, ze w przestrzeni 256 seedów mapowane jest wyłącznie do jednego seeda co zapewnia ze zostanie zwrócone wyłącznie dla hasła które po przekształceniu będzie takie jak prawidłowe, niestety kolizje sa nieuniknione dla tak malej przestrzeni.
# Szyfrowanie Diagram
![](src/honey_szyfrowanie.png)


In [100]:
def encrypt(message, password, message_to_seed, n):
    p_seed = encode_password(password, n)
    m_seed = encode(message, message_to_seed)
    return p_seed ^ m_seed

passwd = 'kryptografia'
message = 'freedom'
cipher = encrypt(message, passwd, message_to_seed, n)
print(f'dla hasła "{message}" zaszyfrowanego kluczem "{passwd}" otrzymamy szyfrogram "{cipher} a binarnie {bin(cipher)}"')

dla hasła "freedom" zaszyfrowanego kluczem "kryptografia" otrzymamy szyfrogram "23 a binarnie 0b10111"


## Funkcja `decrypt(cipher, password, seed_to_message, n)`
Przyjmuje szyfrogram i klucz i zwraca odkodowana wiadomość. dla właściwego klucza i szyfrogramu zaszyfrowanego powyżej zwrócona wiadomość będzie prawidłowa.

# Deszyfrowanie Diagram
![](src/honey_deszyfrowanie.png)

In [91]:
def decrypt(cipher, password, seed_to_message, n):
    p_seed = encode_password(password, n)
    return decode(p_seed ^ cipher, seed_to_message)

decrypted_message = decrypt(cipher, passwd, seed_to_message, n)
print(f'dla szyfrogramu "{cipher}" oraz klucza "{passwd}" odkodowana wiadomość to "{decrypted_message}"')


dla szyfrogramu "23" oraz klucza "kryptografia" odkodowana wiadomość to "freedom"


## Esencja działania Honey Encryption
Poniżej pokazane jest że dla złego klucza Funkcja `decrypt` zwraca wiadomości które dla dobrze zaimplementowanej przestrzeni wiadomości sa praktycznie nierozróżnialne dla atakującego od właściwej wiadomości.
Jest to podstawa działania honey encryption i nie pozwala na atak brute force. Warto zaobserwować, ze częstotliwość występujących wiadomości pokrywa sie z prawdopodobieństwem ich wystąpienia zdefiniowanym w liście `prob_list` (więcej o tym poniżej)

In [92]:
for _ in range (20):
    bad_password = random.randint(1,sys.maxsize)
    decrypted_with_wrong_password = decrypt(cipher, bad_password, seed_to_message, n)
    print(decrypted_with_wrong_password)

123456
123456789
superman
12345678
123456
adobe1
password
123456789
123456
222222
1234567890
andrea
abc123
maggie
asdfghjkl
12345678
michael
1234
123456789
123456


## Statystka prawdopodobieństwa zwracanych wiadomości
Jak opisaliśmy w części teoretycznej funkcja `decrypt` powinna zwracać wiadomości dla złego klucza z prawdopodobieństwem określonym w `prob_list` a sama ta lista jest generowana dla każdej wiadomości i określone prawdopodobieństwo każdej wystąpienia wiadomości powinno byc jak najbardziej zbliżone do rzeczywistego ponieważ na tym opiera się bezpieczeństwo szyfru. Pożądane jest aby prawdopodobieństwo U(P)/G(P) wynosiło w przybliżeniu 1 gdzie U(P) to prawdopodobieństwo ze użytkownik wpisze wiadomość P a G(P) to prawdopodobieństwo ze generator wygeneruje wiadomość P. Poniżej przedstawione jest popróbowanie częstości występowania haseł zmierzonej dla złych kluczy a częstością określona w `prob_list`. Jak widać są one bardzo zbliżone co jest zamierzonym rezultatem. Niestety dla 100000 kluczy i przestrzeni 256 seedów wystąpią liczne kolizje i prawidłowe hasło zostanie zwrócone dla złego klucza

In [93]:
c = 100000
counted = Counter((decrypt(cipher, random.randint(1,sys.maxsize), seed_to_message, n) for _ in range(c)))
prob_list_counted = list()
for passwd, count in sorted(counted.items(), key=lambda x: x[1], reverse=True):
    prob_list_counted.append((passwd, count/c))

zipped = list(zip(prob_list, prob_list_counted))

for p, pc in zipped:
    print(f'hasło - {p[0]}\tprawdziwe prawd. - {p[1]}\tobliczone prawd. - {pc[1]}')

hasło - 123456	prawdziwe prawd. - 0.16876664151602264	obliczone prawd. - 0.17279
hasło - 123456789	prawdziwe prawd. - 0.08234487145110903	obliczone prawd. - 0.08209
hasło - password	prawdziwe prawd. - 0.07188386638148746	obliczone prawd. - 0.06863
hasło - adobe123	prawdziwe prawd. - 0.04399471212905398	obliczone prawd. - 0.04331
hasło - 12345678	prawdziwe prawd. - 0.04189972583719427	obliczone prawd. - 0.04242
hasło - qwerty	prawdziwe prawd. - 0.027194289764519302	obliczone prawd. - 0.0269
hasło - 1234567	prawdziwe prawd. - 0.02582680144086169	obliczone prawd. - 0.02648
hasło - 111111	prawdziwe prawd. - 0.023671536745922372	obliczone prawd. - 0.02307
hasło - photoshop	prawdziwe prawd. - 0.017337523721630176	obliczone prawd. - 0.01624
hasło - 123123	prawdziwe prawd. - 0.017188490566429918	obliczone prawd. - 0.01585
hasło - 1234567890	prawdziwe prawd. - 0.015986248209835355	obliczone prawd. - 0.01553
hasło - 000000	prawdziwe prawd. - 0.015835760058698693	obliczone prawd. - 0.01548
hasło 